In [2]:
#ConversationBufferMemory -> 텍스트를 자동완성 하고싶을때 유용 // 챗봇에서는 별로 .. 대화가 길어질수록 내용이 계속 쌓임
#ConversationBufferWindowMemory -> 대화버퍼메모리는 대화의 특정부분만 저장 // 예를들어 제일최근 5개만 저장하는 방식(유저가 정할 수 있음)) // 단점은 챗봇이 예전 대화를 기억하기 어려움

from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4,
)

def add_message(input, output):
    memory.save_context({"input":input}, {"output":output})

add_message(1,1)
add_message(2,2)
add_message(3,3)
add_message(4,4)

memory.load_memory_variables({})

{'history': [HumanMessage(content='1'),
  AIMessage(content='1'),
  HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4')]}

In [3]:
#ConversationSummaryMemory 형태의 메모리는 시간이 지남에 따라 대화의 요약을 만듭니다. 
# 이것은 시간이 지남에 따라 대화의 정보를 압축하는 데 유용할 수 있습니다. 대화 요약 메모리는 대화가 진행되는 대로 요약하고 현재 요약 내용을 메모리에 저장합니다. 
# 그러면 이 메모리를 사용하여 지금까지의 대화의 요약 내용을 프롬프트/체인에 삽입할 수 있습니다. 이 메모리는 과거 메시지의 기록을 프롬프트 축어적 보고에 유지하는 것이 토큰을 너무 많이 차지하는 긴 대화에 가장 유용합니다.

from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm kyungmin, I live in South Korea", "Wow that is so cool!")

get_history()

{'history': "The human introduces himself as Nicolas from South Korea. The AI responds by expressing admiration for Nicolas's location."}

In [4]:
# ConversationSummaryBufferMemory는 두 가지 아이디어를 결합한 것입니다. 
# 메모리에 최근 상호 작용의 버퍼를 저장하고 있지만 오래된 상호 작용을 완전히 플러싱하는 것이 아니라 요약으로 컴파일하여 두 가지를 모두 사용합니다. 
# 상호 작용을 플러싱할 시점을 결정하기 위해 상호 작용의 수가 아니라 토큰 길이를 사용합니다.

from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm kyungmin, I live in South Korea", "Wow that is so cool!")
get_history()

In [5]:
# Conversation Knowledge Graph : 대화 내용 엔티티의 KG(knowledge graph)를 만들어 중요한 요약본을 생성하여 메모리에 저장하는 방식

from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


add_message("Hi I'm kyungmin, I live in South Korea", "Wow that is so cool!")
memory.load_memory_variables({"input": "who is kyungmin"})

add_message("kyungmin likes kimchi", "Wow that is so cool!")

memory.load_memory_variables({"inputs": "what does kyungmin like"})

{'history': [SystemMessage(content='On kyungmin: kyungmin likes kimchi.')]}

In [13]:
# memory를 chain에 연결
# LLM chain = off the shelf

from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
)

template = """
    You are a helpful AI talking to a human.

    {chat_history}
    Human:{question}
    You:
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
#   verbose=True,
)

chain.predict(question="My name is kyungmin")
chain.predict(question="I live in Seoul")

memory.load_memory_variables({})



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    
    Human:My name is kyungmin
    You:


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is kyungmin
AI: Nice to meet you, Kyungmin! How can I assist you today?
    Human:I live in Seoul
    You:


> Finished chain.


{'chat_history': "Human: My name is kyungmin\nAI: Nice to meet you, Kyungmin! How can I assist you today?\nHuman: I live in Seoul\nAI: That's great to hear! How can I assist you with information or tasks related to Seoul?"}

In [14]:
# Langchain은 off the shelfChain이 있는데 자동으로 LLM으로부터 응답값을 가져오고 memory를 업데이트 하는 것
# 위 같은 방법으로 하면 memory의 기록들을 프롬프트에 넣어줘야했음. langchain expression언어를 활용해 스스로 하게 해주는 방법 처리


# 메모리 추가 3가지 방법 : LLM chain, chat prompt template활용, 여기에있는 수동으로 메모리 관리


from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"] # 수동으로 하면 여기부분을 데이터베이스에서 불러와서 활용할수도 있다.

# 메모리를 먼저 로드하고 프롬프트에 삽입(history)
chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)


invoke_chain("My name is Kyungmin")

invoke_chain("What is my name?")

content='안녕하세요, 강이주님! 무엇을 도와드릴까요?'
content='Your name is 강이주 (Kang Iju). How can I assist you today?'
